I had a problem with the InstructOR model that was difficult to diagnose so I created this toy example to check each step

In [1]:
import pandas as pd
raw_data = pd.read_csv('./inputdata/Mental_Health_FAQ_small.csv')
raw_data.shape

(2, 3)

In [2]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


e:\Code\kaggle\mentalhealth\env\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [31]:
import create_vector_db
from create_vector_db import *

from importlib import reload
reload(create_vector_db)
from create_vector_db import *

dbf = "./db_instructor_large_full_answers_small"
my_model = InstructorLargeEmbeddingModel(model,dbf)


Using embedded DuckDB with persistence: data will be stored in: ./db_instructor_large_full_answers_small


running on GPU


Perform a manual cosine similarity score benchmark using the class functions

In [48]:
query_embeddings = []
for i in range(0, len(raw_data)):
    query_embeddings.append(my_model.embed_query(raw_data['Questions'][i]))

corpus_embeddings = []
for i in range(0, len(raw_data)):
    corpus_embeddings.append(my_model.embed(raw_data['Answers'][i]))

corpus_embeddings_as_nparray = np.array(corpus_embeddings)

manual_similarity_scores = []
for query_embedding in query_embeddings:
    manual_similarity_scores.append(cosine_similarity(np.array(query_embedding).reshape(1,768), corpus_embeddings_as_nparray))

manual_similarity_scores[1]

array([[0.88055338, 0.91992657]])

Now use the code to do the same thing.  
Step 1) Create the ChromaDB

In [9]:
my_model.embed_and_save_knowledge_base(raw_data, 'Answers')

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


Step 2) Since we have embedded the Answers, let's pick a FAQ and see if the database query returns the correct Entry

In [46]:
question = raw_data['Questions'][1]
ans=my_model.search_collection_using_text(search_on = 'Answers', text = question, num_docs=2)

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


Add the manual similarity score and check by ensuring the sum is 1.0

In [49]:
ans["Manual_Similarity_Score"] = [cosine_similarity(np.array(my_model.embed_query(question)).reshape(1,768), np.array(my_model.embed(ans["Answers"][i])).reshape(1,768))[0][0] for i in range(len(ans))]
ans["Total_Similarity_Score"] = ans["Manual_Similarity_Score"] + ans["Scores"]
ans

,Questions,Answers,Scores,Manual_Similarity_Score,Total_Similarity_Score
0,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,0.080073,0.919927,1.0
1,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...,0.119447,0.880553,1.0
